In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
#os.chdir('C:/Users/adtvr/OneDrive/Documents/UTA/TFlowProjects')

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [3]:
def mp_detections(image, model):
    image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results=model.process(image)
    image.flags.writeable= True
    image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image,results

In [4]:
def draw_tracking_points(image, results):
    mp_drawing.draw_landmarks(
        image, 
        results.pose_landmarks,
        mp_holistic.POSE_CONNECTIONS,
        mp_drawing.DrawingSpec(
            color=(0,0,0),
            circle_radius=2
        ),
        mp_drawing.DrawingSpec(
            color=(255,255,255),
            circle_radius=2
        )
    )
    mp_drawing.draw_landmarks(
        image, 
        results.right_hand_landmarks,
        mp_holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(
            color=(0,0,255),
            thickness=1,
            circle_radius=1
        ),
         mp_drawing.DrawingSpec(
            color=(10,10,10),
            thickness=1,
            circle_radius=1
        )
    )
    mp_drawing.draw_landmarks(
        image, 
        results.left_hand_landmarks,
        mp_holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(
            color=(0,0,255),
            thickness=1,
            circle_radius=1
        ),
        mp_drawing.DrawingSpec(
            color=(10,10,10),
            thickness=1,
            circle_radius=1
        )
    )


In [5]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    #face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,lh,rh])

In [6]:
DATA_PATH = os.path.join('ASL_Data') 

actions = np.array(["Hello", "Thank You", "I love you", "Zyzz"])

num_sequences = 30

num_frames = 30

start_folder = 30

In [7]:
for action in actions: 
    for sequence in range(num_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [22]:
cap=cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.7, min_tracking_confidence=0.7) as holistic:
    while cap.isOpened():
        for action in actions:
            for sequence in range(num_sequences):
                for frame_num in range(num_frames):
                                  
                    ret, frame = cap.read()
                    #frame=cv2.flip(frame,1)
                    image, results = mp_detections(frame, holistic)

                    draw_tracking_points(image, results)
                                  
                    if frame_num == 0:
                        cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                        cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                        cv2.imshow('Capturing', image)
                        cv2.waitKey(500)
                        
                    else:
                        cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                        cv2.imshow('Capturing', image)

                    
                    np_tracking_points=extract_keypoints(results)
                    npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                    np.save(npy_path, np_tracking_points)
                    
                                  
                    if cv2.waitKey(10) & 0xFF == ord('q') :
                        break
        cap.release()
        cv2.destroyAllWindows()

In [64]:
cap.release()
cv2.destroyAllWindows()

In [28]:
#import scikitlearn as sklearn
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [29]:
label_map = {label:num for num, label in enumerate(actions)}

In [31]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(num_frames):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [32]:
np.array(sequences).shape

(120, 30, 258)

In [33]:
np.array(labels).shape

(120,)

In [37]:
X = np.array(sequences)
X.shape

(120, 30, 258)

In [38]:
y = to_categorical(labels).astype(int)

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [40]:
y_test.shape

(6, 4)

In [41]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [42]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [46]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,258)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [47]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [48]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
4/4 [==============================] - 4s 251ms/step - loss: 1.3639 - categorical_accuracy: 0.2807
Epoch 2/2000
4/4 [==============================] - 1s 252ms/step - loss: 1.2161 - categorical_accuracy: 0.4386
Epoch 3/2000
4/4 [==============================] - 1s 295ms/step - loss: 1.0265 - categorical_accuracy: 0.5175
Epoch 4/2000
4/4 [==============================] - 1s 255ms/step - loss: 1.0223 - categorical_accuracy: 0.5439
Epoch 5/2000
4/4 [==============================] - 1s 282ms/step - loss: 0.8911 - categorical_accuracy: 0.6404
Epoch 6/2000
4/4 [==============================] - 1s 302ms/step - loss: 0.6911 - categorical_accuracy: 0.7456
Epoch 7/2000
4/4 [==============================] - 1s 264ms/step - loss: 1.1029 - categorical_accuracy: 0.5614
Epoch 8/2000
4/4 [==============================] - 1s 277ms/step - loss: 0.8078 - categorical_accuracy: 0.6667
Epoch 9/2000
4/4 [==============================] - 1s 289ms/step - loss: 0.7473 - categorical_accuracy:

4/4 [==============================] - 1s 292ms/step - loss: 0.6376 - categorical_accuracy: 0.7368
Epoch 74/2000
4/4 [==============================] - 1s 288ms/step - loss: 0.6577 - categorical_accuracy: 0.7456
Epoch 75/2000
4/4 [==============================] - 1s 310ms/step - loss: 0.7427 - categorical_accuracy: 0.6491
Epoch 76/2000
4/4 [==============================] - 1s 280ms/step - loss: 0.5973 - categorical_accuracy: 0.7982
Epoch 77/2000
4/4 [==============================] - 1s 303ms/step - loss: 0.5486 - categorical_accuracy: 0.7982
Epoch 78/2000
4/4 [==============================] - 1s 253ms/step - loss: 0.4945 - categorical_accuracy: 0.7807
Epoch 79/2000
4/4 [==============================] - 1s 295ms/step - loss: 0.3546 - categorical_accuracy: 0.8772
Epoch 80/2000
4/4 [==============================] - 1s 283ms/step - loss: 0.2892 - categorical_accuracy: 0.9298
Epoch 81/2000
4/4 [==============================] - 1s 280ms/step - loss: 0.4649 - categorical_accuracy: 0.88

4/4 [==============================] - 1s 286ms/step - loss: 9.3808 - categorical_accuracy: 0.3509
Epoch 146/2000
4/4 [==============================] - 1s 329ms/step - loss: 1.0229 - categorical_accuracy: 0.5702
Epoch 147/2000
4/4 [==============================] - 1s 245ms/step - loss: 1.0692 - categorical_accuracy: 0.4474
Epoch 148/2000
4/4 [==============================] - 1s 253ms/step - loss: 1.0740 - categorical_accuracy: 0.4561
Epoch 149/2000
4/4 [==============================] - 1s 273ms/step - loss: 1.0277 - categorical_accuracy: 0.5702
Epoch 150/2000
4/4 [==============================] - 1s 307ms/step - loss: 0.9756 - categorical_accuracy: 0.5965
Epoch 151/2000
4/4 [==============================] - 1s 303ms/step - loss: 0.9441 - categorical_accuracy: 0.6316
Epoch 152/2000
4/4 [==============================] - 1s 282ms/step - loss: 0.9133 - categorical_accuracy: 0.6316
Epoch 153/2000
4/4 [==============================] - 1s 295ms/step - loss: 0.8769 - categorical_accura

Epoch 217/2000
4/4 [==============================] - 1s 313ms/step - loss: 0.0530 - categorical_accuracy: 0.9825
Epoch 218/2000
4/4 [==============================] - 1s 333ms/step - loss: 0.1499 - categorical_accuracy: 0.9386
Epoch 219/2000
4/4 [==============================] - 1s 287ms/step - loss: 0.1188 - categorical_accuracy: 0.9649
Epoch 220/2000
4/4 [==============================] - 1s 323ms/step - loss: 0.1039 - categorical_accuracy: 0.9561
Epoch 221/2000
4/4 [==============================] - 1s 298ms/step - loss: 0.0977 - categorical_accuracy: 0.9649
Epoch 222/2000
4/4 [==============================] - 1s 278ms/step - loss: 0.0686 - categorical_accuracy: 0.9825
Epoch 223/2000
4/4 [==============================] - 1s 329ms/step - loss: 0.0760 - categorical_accuracy: 0.9737
Epoch 224/2000
4/4 [==============================] - 1s 286ms/step - loss: 0.0994 - categorical_accuracy: 0.9737
Epoch 225/2000
4/4 [==============================] - 1s 295ms/step - loss: 0.0791 - cat

4/4 [==============================] - 1s 314ms/step - loss: 0.0412 - categorical_accuracy: 0.9737
Epoch 289/2000
4/4 [==============================] - 1s 271ms/step - loss: 0.8245 - categorical_accuracy: 0.8421
Epoch 290/2000
4/4 [==============================] - 1s 281ms/step - loss: 0.6234 - categorical_accuracy: 0.8596
Epoch 291/2000
4/4 [==============================] - 1s 353ms/step - loss: 0.6746 - categorical_accuracy: 0.6754
Epoch 292/2000
4/4 [==============================] - 1s 297ms/step - loss: 0.2756 - categorical_accuracy: 0.9298
Epoch 293/2000
4/4 [==============================] - 1s 290ms/step - loss: 0.2634 - categorical_accuracy: 0.9035
Epoch 294/2000
4/4 [==============================] - 1s 297ms/step - loss: 0.2173 - categorical_accuracy: 0.9123
Epoch 295/2000
4/4 [==============================] - 1s 264ms/step - loss: 0.2742 - categorical_accuracy: 0.9211
Epoch 296/2000
4/4 [==============================] - 1s 286ms/step - loss: 0.2045 - categorical_accura

...

4/4 [==============================] - 1s 279ms/step - loss: 2.6675e-06 - categorical_accuracy: 1.0000
Epoch 1682/2000
4/4 [==============================] - 1s 301ms/step - loss: 2.6539e-06 - categorical_accuracy: 1.0000
Epoch 1683/2000
4/4 [==============================] - 1s 330ms/step - loss: 2.6456e-06 - categorical_accuracy: 1.0000
Epoch 1684/2000
4/4 [==============================] - 1s 298ms/step - loss: 2.6498e-06 - categorical_accuracy: 1.0000
Epoch 1685/2000
4/4 [==============================] - 1s 325ms/step - loss: 2.6351e-06 - categorical_accuracy: 1.0000
Epoch 1686/2000
4/4 [==============================] - 1s 342ms/step - loss: 2.6226e-06 - categorical_accuracy: 1.0000
Epoch 1687/2000
4/4 [==============================] - 1s 304ms/step - loss: 2.6017e-06 - categorical_accuracy: 1.0000
Epoch 1688/2000
4/4 [==============================] - 1s 360ms/step - loss: 2.5933e-06 - categorical_accuracy: 1.0000
Epoch 1689/2000
4/4 [==============================] - 1s 306ms/

Epoch 1750/2000
4/4 [==============================] - 1s 287ms/step - loss: 2.0799e-06 - categorical_accuracy: 1.0000
Epoch 1751/2000
4/4 [==============================] - 1s 282ms/step - loss: 2.0736e-06 - categorical_accuracy: 1.0000
Epoch 1752/2000
4/4 [==============================] - 1s 300ms/step - loss: 2.0652e-06 - categorical_accuracy: 1.0000
Epoch 1753/2000
4/4 [==============================] - 1s 278ms/step - loss: 2.0548e-06 - categorical_accuracy: 1.0000
Epoch 1754/2000
4/4 [==============================] - 1s 342ms/step - loss: 2.0548e-06 - categorical_accuracy: 1.0000
Epoch 1755/2000
4/4 [==============================] - 1s 307ms/step - loss: 2.0537e-06 - categorical_accuracy: 1.0000
Epoch 1756/2000
4/4 [==============================] - 1s 271ms/step - loss: 2.0443e-06 - categorical_accuracy: 1.0000
Epoch 1757/2000
4/4 [==============================] - 1s 288ms/step - loss: 2.0297e-06 - categorical_accuracy: 1.0000
Epoch 1758/2000
4/4 [===========================

4/4 [==============================] - 1s 287ms/step - loss: 1.6553e-06 - categorical_accuracy: 1.0000
Epoch 1819/2000
4/4 [==============================] - 1s 274ms/step - loss: 1.6470e-06 - categorical_accuracy: 1.0000
Epoch 1820/2000
4/4 [==============================] - 1s 286ms/step - loss: 1.6438e-06 - categorical_accuracy: 1.0000
Epoch 1821/2000
4/4 [==============================] - 1s 281ms/step - loss: 1.6375e-06 - categorical_accuracy: 1.0000
Epoch 1822/2000
4/4 [==============================] - 1s 286ms/step - loss: 1.6344e-06 - categorical_accuracy: 1.0000
Epoch 1823/2000
4/4 [==============================] - 1s 274ms/step - loss: 1.6281e-06 - categorical_accuracy: 1.0000
Epoch 1824/2000
4/4 [==============================] - 1s 261ms/step - loss: 1.6250e-06 - categorical_accuracy: 1.0000
Epoch 1825/2000
4/4 [==============================] - 1s 253ms/step - loss: 1.6271e-06 - categorical_accuracy: 1.0000
Epoch 1826/2000
4/4 [==============================] - 1s 277ms/

Epoch 1887/2000
4/4 [==============================] - 1s 322ms/step - loss: 1.3061e-06 - categorical_accuracy: 1.0000
Epoch 1888/2000
4/4 [==============================] - 1s 323ms/step - loss: 1.3029e-06 - categorical_accuracy: 1.0000
Epoch 1889/2000
4/4 [==============================] - 1s 323ms/step - loss: 1.3019e-06 - categorical_accuracy: 1.0000
Epoch 1890/2000
4/4 [==============================] - 1s 284ms/step - loss: 1.2987e-06 - categorical_accuracy: 1.0000
Epoch 1891/2000
4/4 [==============================] - 1s 340ms/step - loss: 1.2956e-06 - categorical_accuracy: 1.0000
Epoch 1892/2000
4/4 [==============================] - 1s 316ms/step - loss: 1.2914e-06 - categorical_accuracy: 1.0000
Epoch 1893/2000
4/4 [==============================] - 1s 273ms/step - loss: 1.2799e-06 - categorical_accuracy: 1.0000
Epoch 1894/2000
4/4 [==============================] - 1s 300ms/step - loss: 1.2757e-06 - categorical_accuracy: 1.0000
Epoch 1895/2000
4/4 [===========================

4/4 [==============================] - 1s 275ms/step - loss: 1.0300e-06 - categorical_accuracy: 1.0000
Epoch 1956/2000
4/4 [==============================] - 1s 297ms/step - loss: 1.0269e-06 - categorical_accuracy: 1.0000
Epoch 1957/2000
4/4 [==============================] - 1s 308ms/step - loss: 1.0279e-06 - categorical_accuracy: 1.0000
Epoch 1958/2000
4/4 [==============================] - 1s 345ms/step - loss: 1.0248e-06 - categorical_accuracy: 1.0000
Epoch 1959/2000
4/4 [==============================] - 1s 269ms/step - loss: 1.0175e-06 - categorical_accuracy: 1.0000
Epoch 1960/2000
4/4 [==============================] - 1s 305ms/step - loss: 1.0101e-06 - categorical_accuracy: 1.0000
Epoch 1961/2000
4/4 [==============================] - 1s 314ms/step - loss: 1.0185e-06 - categorical_accuracy: 1.0000
Epoch 1962/2000
4/4 [==============================] - 1s 291ms/step - loss: 1.0060e-06 - categorical_accuracy: 1.0000
Epoch 1963/2000
4/4 [==============================] - 1s 289ms/

In [49]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            82688     
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 4)                 132       
                                                                 
Total params: 237,284
Trainable params: 237,284
Non-tr

In [51]:
res = model.predict(X_test)

1/1 [==============================] - 0s 411ms/step


In [52]:
actions[np.argmax(res[4])]

'I love you'

In [53]:
actions[np.argmax(y_test[4])]

'I love you'

In [54]:
model.save('action.h5')

In [55]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [56]:
yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
multilabel_confusion_matrix(ytrue, yhat)

1/1 [==============================] - 0s 73ms/step


array([[[3, 0],
        [0, 3]],

       [[4, 0],
        [0, 2]],

       [[5, 0],
        [0, 1]]], dtype=int64)

In [57]:
accuracy_score(ytrue, yhat)

1.0

In [58]:
from scipy import stats

In [65]:
colors = [(245,117,16), (117,245,16), (16,117,245), (17,29,0)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [74]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mp_detections(frame, holistic)
        
        # Draw landmarks
        draw_tracking_points(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            #image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 35ms/step
Hello
1/1 [==============================] - 0s 33ms/step
Hello
1/1 [==============================] - 0s 36ms/step
Hello
1/1 [==============================] - 0s 38ms/step
Hello
1/1 [==============================] - 0s 35ms/step
Hello
1/1 [==============================] - 0s 34ms/step
Hello
1/1 [==============================] - 0s 32ms/step
Hello
1/1 [==============================] - 0s 33ms/step
Hello
1/1 [==============================] - 0s 34ms/step
Hello
1/1 [==============================] - 0s 35ms/step
Hello
1/1 [==============================] - 0s 33ms/step
Hello
1/1 [==============================] - 0s 34ms/step
Hello
1/1 [==============================] - 0s 35ms/step
Hello
1/1 [==============================] - 0s 36ms/step
Hello
1/1 [==============================] - 0s 33ms/step
Hello
1/1 [==============================] - 0s 33ms/step
Hello
1/1 [==============================] - 0s 36ms/step
Hello
1/1 [=========

Zyzz
1/1 [==============================] - 0s 35ms/step
Zyzz
1/1 [==============================] - 0s 34ms/step
Zyzz
1/1 [==============================] - 0s 33ms/step
Zyzz
1/1 [==============================] - 0s 33ms/step
I love you
1/1 [==============================] - 0s 34ms/step
I love you
1/1 [==============================] - 0s 34ms/step
I love you
1/1 [==============================] - 0s 33ms/step
I love you
1/1 [==============================] - 0s 34ms/step
Hello
1/1 [==============================] - 0s 34ms/step
Hello
1/1 [==============================] - 0s 34ms/step
Hello
1/1 [==============================] - 0s 35ms/step
Hello
1/1 [==============================] - 0s 33ms/step
Hello
1/1 [==============================] - 0s 34ms/step
Hello
1/1 [==============================] - 0s 33ms/step
Hello
1/1 [==============================] - 0s 34ms/step
Hello
1/1 [==============================] - 0s 34ms/step
Hello
1/1 [==============================] - 0s 34ms/ste